In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
%matplotlib inline

In [2]:
path = os.getcwd().split('TianChi_IJCAI-17_footfall_prediction_code')[0]

shop_info = pd.read_table(path+'dataset/shop_info.txt',names=['shop_id','city_name','location_id','per_pay','score','comment_cnt','shop_level','cate_1_name','cate_2_name','cate_3_name'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)


In [3]:
shop_info.head()

,shop_id,city_name,location_id,per_pay,score,comment_cnt,shop_level,cate_1_name,cate_2_name,cate_3_name
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果


In [4]:
shop_info.describe()

,shop_id,location_id,per_pay,score,comment_cnt,shop_level
count,2000.000000,2000.000000,2000.000000,1709.000000,1709.000000,2000.000000
mean,1000.500000,583.083000,10.479000,2.677004,3.130486,0.814500
std,577.494589,335.763357,5.762479,1.024211,3.499072,0.855836
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,500.750000,287.750000,5.000000,2.000000,1.000000,0.000000
50%,1000.500000,577.500000,10.000000,3.000000,2.000000,1.000000
75%,1500.250000,877.250000,15.000000,3.000000,4.000000,2.000000
max,2000.000000,1159.000000,20.000000,4.000000,20.000000,2.000000


In [5]:
shop_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
shop_id        2000 non-null int64
city_name      2000 non-null object
location_id    2000 non-null int64
per_pay        2000 non-null int64
score          1709 non-null float64
comment_cnt    1709 non-null float64
shop_level     2000 non-null int64
cate_1_name    2000 non-null object
cate_2_name    2000 non-null object
cate_3_name    1415 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 156.3+ KB


In [6]:
shop_info['cate_1_name'].value_counts()

美食          1415
超市便利店        579
医疗健康           2
休闲娱乐           2
美发/美容/美甲       1
购物             1
Name: cate_1_name, dtype: int64

In [7]:
shop_info['cate_2_name'].value_counts()

快餐             639
超市             372
便利店            206
休闲茶饮           177
小吃             156
休闲食品           150
烘焙糕点           122
中餐              78
其他美食            44
火锅              33
烧烤               9
汤/粥/煲/砂锅/炖菜      7
药店               2
网吧网咖             2
美容美发             1
本地购物             1
个人护理             1
Name: cate_2_name, dtype: int64

In [8]:
shop_info['cate_3_name'].value_counts()

西式快餐        405
中式快餐        220
生鲜水果        111
奶茶           92
其它小吃         87
面包           71
饮品/甜点        67
面点           50
蛋糕           39
零食           22
其它地方菜        20
西餐           19
其它休闲食品       16
日韩料理         15
米粉/米线        15
其它快餐         14
江浙菜          13
川味/重庆火锅      13
川菜           13
其它烘焙糕点       12
粤菜           12
其它火锅         11
冰激凌          10
麻辣烫/串串香       9
中式烧烤          8
其他餐饮美食        7
咖啡            6
西北菜           6
熟食            4
砂锅/煲类/炖菜      4
东北菜           3
海鲜            3
自助餐           3
粥             3
台湾菜           2
湖北菜           2
咖啡厅           2
其它烧烤          1
湘菜            1
闽菜            1
香锅/烤鱼         1
上海本帮菜         1
美食特产          1
Name: cate_3_name, dtype: int64

In [9]:

user_pay = pd.read_table(path+'dataset/user_pay.txt',names=['user_id','shop_id','time_stamp'],header=None,delimiter=',',encoding='utf8',delim_whitespace=False,index_col=False)
#user_pay = pd.read_csv('user_pay_part.csv')

In [10]:
#user_pay.iloc[:1000,:].to_csv('user_pay_part.csv',sep=',',index=False)

In [11]:
user_pay.count()

user_id       69674110
shop_id       69674110
time_stamp    69674110
dtype: int64

In [12]:
user_pay.head()

,user_id,shop_id,time_stamp
0,22127870,1862,2015-12-25 17:00:00
1,3434231,1862,2016-10-05 11:00:00
2,16955285,1862,2016-02-10 15:00:00
3,13799128,1862,2016-01-13 14:00:00
4,13799128,1862,2016-07-05 12:00:00


In [13]:
#len(user_pay['user_id'].unique()) #  用户数 user_id

In [14]:

def get_time_year(x):
    return str(x).split('-')[0]

#user_pay['time_year'] = user_pay['time_stamp'].apply(lambda x:get_time_year(x) )


In [15]:
#user_pay['time_year'].value_counts()

In [16]:
#sns.countplot(user_pay['time_year'])

In [17]:
#user_pay['time_stamp'].to_timestamp()

In [18]:
def get_time_month(x):
    return x.split('-')[0] +'-'+ x.split('-')[1]

#user_pay['time_month'] = user_pay['time_stamp'].apply(lambda x:get_time_month(x) )


In [19]:
#plt.figure(figsize =(14,10))
#month_order = '2015-07','2015-08','2015-09','2015-10','2015-11','2015-12','2016-01','2016-02','2016-03','2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10']
#sns.countplot(user_pay['time_month'],order=month_order)

In [20]:
#pd.to_datetime(user_pay['time_stamp'].tolist())

In [21]:
time_index = pd.to_datetime(user_pay['time_stamp'].tolist())

In [22]:
print type(time_index)

<class 'pandas.tseries.index.DatetimeIndex'>


In [23]:
user_pay.index = time_index

In [24]:
user_pay.head()

,user_id,shop_id,time_stamp
2015-12-25 17:00:00,22127870,1862,2015-12-25 17:00:00
2016-10-05 11:00:00,3434231,1862,2016-10-05 11:00:00
2016-02-10 15:00:00,16955285,1862,2016-02-10 15:00:00
2016-01-13 14:00:00,13799128,1862,2016-01-13 14:00:00
2016-07-05 12:00:00,13799128,1862,2016-07-05 12:00:00


In [25]:
len(user_pay)

69674110

In [26]:
def dateRange(start, end):
    days = (datetime.datetime.strptime(end, "%Y-%m-%d") - datetime.datetime.strptime(start, "%Y-%m-%d")).days + 1
    return [datetime.datetime.strftime(datetime.datetime.strptime(start, "%Y-%m-%d") + datetime.timedelta(i), "%Y-%m-%d") for i in xrange(days)]
#len(dateRange('2015-07-01','2016-10-31'))

In [27]:
start_time = time.time()
def get_shop_sales_count_each_day(x,day):
    if x % 100 == 0:
        print 'day = ',day,'id = ',x
    return len(user_pay[user_pay['shop_id'] == x][day])

for day in dateRange('2015-07-01','2016-10-31'):
    shop_info[day] = shop_info['shop_id'].apply(lambda x: get_shop_sales_count_each_day(x,day))
    #print 'day = ',day
end_time = time.time()
print 'cost ',end_time-start_time,' seconds'
#  估计用时为 489 × 166/3600 = 22.4h 实际用时 18.7h

day =  2015-07-01 id =  100
day =  2015-07-01 id =  200
day =  2015-07-01 id =  300
day =  2015-07-01 id =  400
day =  2015-07-01 id =  500
day =  2015-07-01 id =  600
day =  2015-07-01 id =  700
day =  2015-07-01 id =  800
day =  2015-07-01 id =  900
day =  2015-07-01 id =  1000
day =  2015-07-01 id =  1100
day =  2015-07-01 id =  1200
day =  2015-07-01 id =  1300
day =  2015-07-01 id =  1400
day =  2015-07-01 id =  1500
day =  2015-07-01 id =  1600
day =  2015-07-01 id =  1700
day =  2015-07-01 id =  1800
day =  2015-07-01 id =  1900
day =  2015-07-01 id =  2000
day =  2015-07-02 id =  100
day =  2015-07-02 id =  200
day =  2015-07-02 id =  300
day =  2015-07-02 id =  400
day =  2015-07-02 id =  500
day =  2015-07-02 id =  600
day =  2015-07-02 id =  700
day =  2015-07-02 id =  800
day =  2015-07-02 id =  900
day =  2015-07-02 id =  1000
day =  2015-07-02 id =  1100
day =  2015-07-02 id =  1200
day =  2015-07-02 id =  1300
day =  2015-07-02 id =  1400
day =  2015-07-02 id =  1500
day

In [28]:
def save_shop_sales_count_each_day():
    shop_info.to_csv(path+'dataset/shop info with count.csv',sep=',',index=False,encoding='utf-8')
#save_shop_sales_count_each_day()  

In [29]:
def save_shop_sales_count_each_day_separately():
    shop_info.loc[:,dateRange('2015-07-01','2016-10-31')].to_csv(path+'dataset/shop count separately.csv',sep=',',index=False,encoding='utf-8')
#save_shop_sales_count_each_day_separately()  

start_time = time.time()
global count 
count = 0
def get_shop_sales_count_each_day2(x):
    shop_info.loc[shop_info['shop_id']==x['shop_id'],shop_info['time_stamp'.split(' ')[0]]] += 1
    #if count % 100 == 0 :
    print count
    count +=1             
for day in dateRang                  
e('2015-07-01','2016-10-31'):
    shop_info[day] = 0
user_pay.apply(lambda x: get_shop_sales_count_each_day2(x),axis=1 )
end_time = time.time()
print 'cost ',end_time-start_time,' seconds'